In [65]:
import geopandas as gpd
import os 
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from unidecode import unidecode

In [66]:
#LIMITE_DEPARTAMENTAL_INEI_geogpsperu
os.chdir('C:/Users/FERNANDO/Documents/PI_INEQUIDAD/scripts_data/data/inputs/shape_files/GPK/Cap_Provincia')
gdf = gpd.read_file('Cap_Provincia.SHP')
#gdf_1=gdf.iloc[:,[0,4,5,8]].copy()
#gdf_1
puntos_cap=gdf.to_crs(gdf.crs)

In [67]:
puntos_cap['Departa'].unique() # esto muestra que si se registran todas las regiones incluido el callao
puntos_cap_sort = puntos_cap.sort_values(by='Departa')
puntos_cap_sort

,Cod_IGN,Rasgo_Prin,Categoria,Nombre,Provincia,Departa,X,Y,Cod_Provin,Cod_Depart,geometry
174,CO230,Capital,VILLA,SANTA MARIA DE NIEVA,CONDORCANQUI,AMAZONAS,-77.86752,-4.59404,0104,01,POINT (-77.86752 -4.59404)
164,CO230,Capital,CIUDAD,BAGUA GRANDE,UTCUBAMBA,AMAZONAS,-78.44228,-5.75506,0107,01,POINT (-78.44228 -5.75506)
163,CO230,Capital,VILLA,BAGUA,BAGUA,AMAZONAS,-78.43743,-5.61002,0102,01,POINT (-78.53869 -5.62955)
161,CO230,Capital,VILLA,JUMBILLA,BONGARÁ,AMAZONAS,-77.79888,-5.90643,0103,01,POINT (-77.79888 -5.90643)
155,CO230,Capital,CIUDAD,MENDOZA,RODRIGUEZ DE MENDOZA,AMAZONAS,-77.48190,-6.39411,0106,01,POINT (-77.4819 -6.39411)
...,...,...,...,...,...,...,...,...,...,...,...
183,CO230,Capital,CIUDAD,ZARUMILLA,ZARUMILLA,TUMBES,-80.27329,-3.49904,2403,24,POINT (-80.27329 -3.49904)
12,CO230,Capital,VILLA,ATALAYA,ATALAYA,UCAYALI,-73.75576,-10.72740,2502,25,POINT (-73.75576 -10.7274)
105,CO230,Capital,VILLA,AGUAYTÍA,PADRE ABAD,UCAYALI,-75.51013,-9.03709,2503,25,POINT (-75.51012 -9.03709)
127,CO230,Capital,CIUDAD,PUCALLPA,CORONEL PORTILLO,UCAYALI,-74.53067,-8.37881,2501,25,POINT (-74.53067 -8.37881)


In [68]:
from shapely.geometry import Point
nueva_fila = {
    'Cod_IGN': 'CO230',
    'Rasgo_Prin': 'Capital',
    'Categoria': 'CIUDAD',
    'Nombre': 'SAN ANTONIO DEL ESTRECHO',
    'Provincia': 'PUTUMAYO',
    'Departa': 'LORETO',
    'X': -72.66846,
    'Y': -2.45028,
    'Cod_Provin': 1607,
    'Cod_Depart': 16,
    'geometry': Point(-72.66846, -2.45028)
}

# Crear un nuevo GeoDataFrame con esa fila
nueva_gdf = gpd.GeoDataFrame([nueva_fila], columns=gdf.columns, crs=gdf.crs)

# Añadir la nueva fila al GeoDataFrame original

puntos_cap_sort = pd.concat([puntos_cap_sort, nueva_gdf], ignore_index=True)

In [69]:
os.chdir('C:/Users/FERNANDO/Documents/PI_INEQUIDAD/scripts_data/data/inputs/shape_files/capital_dep')

capxprv=pd.read_excel('capxdep.xlsx')

capxprv['Departamento'] = capxprv['Departamento'].str.upper()

capxprv['Capital'] = capxprv['Capital'].str.upper()

capxprv_1=capxprv.iloc[1:,:-1].reset_index(drop=True)
capxprv_1=capxprv_1.dropna()
capxprv.dtypes

n_ubbig=[]

for elem in capxprv_1['Ubigeo']:
    if int(elem)<10:
        n_ubbig+=['0'+str(int(elem))]
    else:
        n_ubbig+=[str(int(elem))]
n_ubbig

capxprv_1['Ubigeo']=n_ubbig

capxprv_1.reset_index(drop=True)

capxprv_1

,Ubigeo,Departamento,Capital
0,01,AMAZONAS,CHACHAPOYAS
1,02,ÁNCASH,HUARAZ
2,03,APURÍMAC,ABANCAY
3,04,AREQUIPA,AREQUIPA
4,05,AYACUCHO,AYACUCHO
5,06,CAJAMARCA,CAJAMARCA
6,07,CALLAO,CALLAO
7,08,CUSCO,CUZCO
8,09,HUANCAVELICA,HUANCAVELICA
9,10,HUÁNUCO,HUÁNUCO


In [70]:
import time
# Función para obtener coordenadas
def obtener_coordenadas(ciudad, region):
    geolocator = Nominatim(user_agent="peru_capitals")
    location = geolocator.geocode(f"{ciudad}, {region}, Perú")
    if location:
        return (location.latitude, location.longitude)
    return None

capitales_data = []
for idx, row in capxprv_1.iterrows():
    ubig=row['Ubigeo']
    departamento=row['Departamento']
    capital=row['Capital']
    coords = obtener_coordenadas(capital, departamento)
    if coords:
        capitales_data.append({
            "region": departamento,
            "capital": capital,
            "latitud": coords[0],
            "longitud": coords[1]
        })
    time.sleep(1)  # Para evitar sobrecargar el servidor

In [71]:
df_capitales = pd.DataFrame(capitales_data)
df_capitales

,region,capital,latitud,longitud
0,AMAZONAS,CHACHAPOYAS,-6.371827,-77.809338
1,ÁNCASH,HUARAZ,-9.529851,-77.528998
2,APURÍMAC,ABANCAY,-13.637348,-72.878874
3,AREQUIPA,AREQUIPA,-16.398867,-71.536961
4,AYACUCHO,AYACUCHO,-13.160427,-74.225697
5,CAJAMARCA,CAJAMARCA,-7.156668,-78.517306
6,CALLAO,CALLAO,-12.052263,-77.139113
7,CUSCO,CUZCO,-13.516954,-71.979271
8,HUANCAVELICA,HUANCAVELICA,-12.787280,-74.973095
9,HUÁNUCO,HUÁNUCO,-9.929325,-76.239485


In [72]:
# SE COMPRUEBA QUE LAS COLUMNAS DE INTERES SON OBJETOS
# PRINCIPALMENTE SE NECESITA LA COLUMNA UBIGEO
# EN ESTA DATA capxpriv voy a generar una columna que tenga registrada los centroides por provincia
puntos_cap_sort.dtypes
#capxprv_1.dtypes

Cod_IGN         object
Rasgo_Prin      object
Categoria       object
Nombre          object
Provincia       object
Departa         object
X              float64
Y              float64
Cod_Provin      object
Cod_Depart      object
geometry      geometry
dtype: object

In [73]:
merged = df_capitales.merge(puntos_cap_sort[['Nombre','Provincia', 'Departa','geometry']], left_on='region', right_on='Departa', how='inner')


# ERROR
---
La data no hace match con lso nombres de todo los departamentos

In [74]:
df_final=merged.iloc[:,[0,1,2,3,5,6]].copy()
df_final


,region,capital,latitud,longitud,Provincia,Departa
0,AMAZONAS,CHACHAPOYAS,-6.371827,-77.809338,CONDORCANQUI,AMAZONAS
1,AMAZONAS,CHACHAPOYAS,-6.371827,-77.809338,UTCUBAMBA,AMAZONAS
2,AMAZONAS,CHACHAPOYAS,-6.371827,-77.809338,BAGUA,AMAZONAS
3,AMAZONAS,CHACHAPOYAS,-6.371827,-77.809338,BONGARÁ,AMAZONAS
4,AMAZONAS,CHACHAPOYAS,-6.371827,-77.809338,RODRIGUEZ DE MENDOZA,AMAZONAS
...,...,...,...,...,...,...
143,TUMBES,TUMBES,-3.570787,-80.459645,ZARUMILLA,TUMBES
144,UCAYALI,PUCALLPA,-8.382074,-74.538751,ATALAYA,UCAYALI
145,UCAYALI,PUCALLPA,-8.382074,-74.538751,PADRE ABAD,UCAYALI
146,UCAYALI,PUCALLPA,-8.382074,-74.538751,CORONEL PORTILLO,UCAYALI


# SOLUCIÓN
---
Le quitamos los tildes a los nombres con unidecode

In [75]:
df_capitales['region']=df_capitales['region'].apply(unidecode)
df_capitales.loc[df_capitales['region'] == 'LIMA METROPOLITANA', 'region'] = 'LIMA' # para remplazar lima metropolitana por lima

In [76]:
puntos_cap_sort['Departa']=puntos_cap_sort['Departa'].apply(unidecode)

In [77]:
puntos_cap_sort['Departa'].unique() # esto muestra que si se registran todas las regiones incluido el callao

array(['AMAZONAS', 'ANCASH', 'APURIMAC', 'AREQUIPA', 'AYACUCHO',
       'CAJAMARCA', 'CALLAO', 'CUSCO', 'HUANCAVELICA', 'HUANUCO', 'ICA',
       'JUNIN', 'LA LIBERTAD', 'LAMBAYEQUE', 'LIMA', 'LORETO',
       'MADRE DE DIOS', 'MOQUEGUA', 'PASCO', 'PIURA', 'PUNO',
       'SAN MARTIN', 'TACNA', 'TUMBES', 'UCAYALI'], dtype=object)

In [78]:
merged = df_capitales.merge(puntos_cap_sort[['Nombre','Provincia', 'Departa','geometry']], left_on='region', right_on='Departa', how='inner')

In [79]:
merged

,region,capital,latitud,longitud,Nombre,Provincia,Departa,geometry
0,AMAZONAS,CHACHAPOYAS,-6.371827,-77.809338,SANTA MARIA DE NIEVA,CONDORCANQUI,AMAZONAS,POINT (-77.86752 -4.59404)
1,AMAZONAS,CHACHAPOYAS,-6.371827,-77.809338,BAGUA GRANDE,UTCUBAMBA,AMAZONAS,POINT (-78.44228 -5.75506)
2,AMAZONAS,CHACHAPOYAS,-6.371827,-77.809338,BAGUA,BAGUA,AMAZONAS,POINT (-78.53869 -5.62955)
3,AMAZONAS,CHACHAPOYAS,-6.371827,-77.809338,JUMBILLA,BONGARÁ,AMAZONAS,POINT (-77.79888 -5.90643)
4,AMAZONAS,CHACHAPOYAS,-6.371827,-77.809338,MENDOZA,RODRIGUEZ DE MENDOZA,AMAZONAS,POINT (-77.4819 -6.39411)
...,...,...,...,...,...,...,...,...
191,TUMBES,TUMBES,-3.570787,-80.459645,ZARUMILLA,ZARUMILLA,TUMBES,POINT (-80.27329 -3.49904)
192,UCAYALI,PUCALLPA,-8.382074,-74.538751,ATALAYA,ATALAYA,UCAYALI,POINT (-73.75576 -10.7274)
193,UCAYALI,PUCALLPA,-8.382074,-74.538751,AGUAYTÍA,PADRE ABAD,UCAYALI,POINT (-75.51012 -9.03709)
194,UCAYALI,PUCALLPA,-8.382074,-74.538751,PUCALLPA,CORONEL PORTILLO,UCAYALI,POINT (-74.53067 -8.37881)


In [80]:
# Calcular centroides
df_final=merged.iloc[:,[0,1,5,4,3,2,7]]
df_final
df_final.rename(columns={'geometry': 'cap_provincia'}, inplace=True)

gdf_final = gpd.GeoDataFrame(
    df_final,
    geometry=gpd.points_from_xy(df_final['longitud'], df_final['latitud']),
    crs="EPSG:4326"  # Sistema de coordenadas WGS84
)

gdf_final

C:\Users\FERNANDO\AppData\Local\Temp\ipykernel_13760\1223094435.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.rename(columns={'geometry': 'cap_provincia'}, inplace=True)


,region,capital,Provincia,Nombre,longitud,latitud,cap_provincia,geometry
0,AMAZONAS,CHACHAPOYAS,CONDORCANQUI,SANTA MARIA DE NIEVA,-77.809338,-6.371827,POINT (-77.86752 -4.59404),POINT (-77.80934 -6.37183)
1,AMAZONAS,CHACHAPOYAS,UTCUBAMBA,BAGUA GRANDE,-77.809338,-6.371827,POINT (-78.44228 -5.75506),POINT (-77.80934 -6.37183)
2,AMAZONAS,CHACHAPOYAS,BAGUA,BAGUA,-77.809338,-6.371827,POINT (-78.53869 -5.62955),POINT (-77.80934 -6.37183)
3,AMAZONAS,CHACHAPOYAS,BONGARÁ,JUMBILLA,-77.809338,-6.371827,POINT (-77.79888 -5.90643),POINT (-77.80934 -6.37183)
4,AMAZONAS,CHACHAPOYAS,RODRIGUEZ DE MENDOZA,MENDOZA,-77.809338,-6.371827,POINT (-77.4819 -6.39411),POINT (-77.80934 -6.37183)
...,...,...,...,...,...,...,...,...
191,TUMBES,TUMBES,ZARUMILLA,ZARUMILLA,-80.459645,-3.570787,POINT (-80.27329 -3.49904),POINT (-80.45965 -3.57079)
192,UCAYALI,PUCALLPA,ATALAYA,ATALAYA,-74.538751,-8.382074,POINT (-73.75576 -10.7274),POINT (-74.53875 -8.38207)
193,UCAYALI,PUCALLPA,PADRE ABAD,AGUAYTÍA,-74.538751,-8.382074,POINT (-75.51012 -9.03709),POINT (-74.53875 -8.38207)
194,UCAYALI,PUCALLPA,CORONEL PORTILLO,PUCALLPA,-74.538751,-8.382074,POINT (-74.53067 -8.37881),POINT (-74.53875 -8.38207)


In [81]:
dt_final=gdf_final.iloc[:,[0,1,2,3,6,7]]
dt_final

,region,capital,Provincia,Nombre,cap_provincia,geometry
0,AMAZONAS,CHACHAPOYAS,CONDORCANQUI,SANTA MARIA DE NIEVA,POINT (-77.86752 -4.59404),POINT (-77.80934 -6.37183)
1,AMAZONAS,CHACHAPOYAS,UTCUBAMBA,BAGUA GRANDE,POINT (-78.44228 -5.75506),POINT (-77.80934 -6.37183)
2,AMAZONAS,CHACHAPOYAS,BAGUA,BAGUA,POINT (-78.53869 -5.62955),POINT (-77.80934 -6.37183)
3,AMAZONAS,CHACHAPOYAS,BONGARÁ,JUMBILLA,POINT (-77.79888 -5.90643),POINT (-77.80934 -6.37183)
4,AMAZONAS,CHACHAPOYAS,RODRIGUEZ DE MENDOZA,MENDOZA,POINT (-77.4819 -6.39411),POINT (-77.80934 -6.37183)
...,...,...,...,...,...,...
191,TUMBES,TUMBES,ZARUMILLA,ZARUMILLA,POINT (-80.27329 -3.49904),POINT (-80.45965 -3.57079)
192,UCAYALI,PUCALLPA,ATALAYA,ATALAYA,POINT (-73.75576 -10.7274),POINT (-74.53875 -8.38207)
193,UCAYALI,PUCALLPA,PADRE ABAD,AGUAYTÍA,POINT (-75.51012 -9.03709),POINT (-74.53875 -8.38207)
194,UCAYALI,PUCALLPA,CORONEL PORTILLO,PUCALLPA,POINT (-74.53067 -8.37881),POINT (-74.53875 -8.38207)


In [82]:
dt_final= dt_final.to_crs('EPSG:32718')
dt_final['cap_provincia'] = dt_final['cap_provincia'].to_crs('EPSG:32718')

dt_final['distancia_kilometros'] = dt_final.apply(
        lambda row: row['cap_provincia'].distance(row['geometry'])/1000,
        axis=1
    )

In [83]:
dt_final

,region,capital,Provincia,Nombre,cap_provincia,geometry,distancia_kilometros
0,AMAZONAS,CHACHAPOYAS,CONDORCANQUI,SANTA MARIA DE NIEVA,POINT (181802.836 9491571.769),POINT (189187.025 9294846.388),196.863917
1,AMAZONAS,CHACHAPOYAS,UTCUBAMBA,BAGUA GRANDE,POINT (118652.319 9362721.449),POINT (189187.025 9294846.388),97.888552
2,AMAZONAS,CHACHAPOYAS,BAGUA,BAGUA,POINT (107872.199 9376556.891),POINT (189187.025 9294846.388),115.276655
3,AMAZONAS,CHACHAPOYAS,BONGARÁ,JUMBILLA,POINT (190075.874 9346359.715),POINT (189187.025 9294846.388),51.520995
4,AMAZONAS,CHACHAPOYAS,RODRIGUEZ DE MENDOZA,MENDOZA,POINT (225448.752 9292567.08),POINT (189187.025 9294846.388),36.333292
...,...,...,...,...,...,...,...
191,TUMBES,TUMBES,ZARUMILLA,ZARUMILLA,POINT (-86526.726 9611595.721),POINT (-107268.982 9603509.416),22.262739
192,UCAYALI,PUCALLPA,ATALAYA,ATALAYA,POINT (636058.908 8813886.747),POINT (550781.38 9073433.117),273.196952
193,UCAYALI,PUCALLPA,PADRE ABAD,AGUAYTÍA,POINT (443935.142 9001007.633),POINT (550781.38 9073433.117),129.079701
194,UCAYALI,PUCALLPA,CORONEL PORTILLO,PUCALLPA,POINT (551671.316 9073793.294),POINT (550781.38 9073433.117),0.960059


In [84]:

dt_final.rename(columns={'geometry': 'cap_region'}, inplace=True)

In [85]:
dt_final # SE HACE UN AJUSTE PARA LAS REGIONES DE LAMBAYEQUE, PUES NO SE ESTAN REGISTRANDO CORRECTAMENTE LAS PROVINICAS
dt_final.loc[125, 'Provincia'] = 'FERRENAFE'
dt_final.loc[126, 'Provincia'] = 'CHICLAYO'
dt_final

,region,capital,Provincia,Nombre,cap_provincia,cap_region,distancia_kilometros
0,AMAZONAS,CHACHAPOYAS,CONDORCANQUI,SANTA MARIA DE NIEVA,POINT (181802.836 9491571.769),POINT (189187.025 9294846.388),196.863917
1,AMAZONAS,CHACHAPOYAS,UTCUBAMBA,BAGUA GRANDE,POINT (118652.319 9362721.449),POINT (189187.025 9294846.388),97.888552
2,AMAZONAS,CHACHAPOYAS,BAGUA,BAGUA,POINT (107872.199 9376556.891),POINT (189187.025 9294846.388),115.276655
3,AMAZONAS,CHACHAPOYAS,BONGARÁ,JUMBILLA,POINT (190075.874 9346359.715),POINT (189187.025 9294846.388),51.520995
4,AMAZONAS,CHACHAPOYAS,RODRIGUEZ DE MENDOZA,MENDOZA,POINT (225448.752 9292567.08),POINT (189187.025 9294846.388),36.333292
...,...,...,...,...,...,...,...
191,TUMBES,TUMBES,ZARUMILLA,ZARUMILLA,POINT (-86526.726 9611595.721),POINT (-107268.982 9603509.416),22.262739
192,UCAYALI,PUCALLPA,ATALAYA,ATALAYA,POINT (636058.908 8813886.747),POINT (550781.38 9073433.117),273.196952
193,UCAYALI,PUCALLPA,PADRE ABAD,AGUAYTÍA,POINT (443935.142 9001007.633),POINT (550781.38 9073433.117),129.079701
194,UCAYALI,PUCALLPA,CORONEL PORTILLO,PUCALLPA,POINT (551671.316 9073793.294),POINT (550781.38 9073433.117),0.960059


In [86]:

dt_final.to_excel("C:/Users/FERNANDO/Documents/PI_INEQUIDAD/scripts_data/data/outputs/distancias/cap_regionalxprovincias/dist_provincia_cap_to_cap_region.xlsx", index=False)
dt_final


,region,capital,Provincia,Nombre,cap_provincia,cap_region,distancia_kilometros
0,AMAZONAS,CHACHAPOYAS,CONDORCANQUI,SANTA MARIA DE NIEVA,POINT (181802.836 9491571.769),POINT (189187.025 9294846.388),196.863917
1,AMAZONAS,CHACHAPOYAS,UTCUBAMBA,BAGUA GRANDE,POINT (118652.319 9362721.449),POINT (189187.025 9294846.388),97.888552
2,AMAZONAS,CHACHAPOYAS,BAGUA,BAGUA,POINT (107872.199 9376556.891),POINT (189187.025 9294846.388),115.276655
3,AMAZONAS,CHACHAPOYAS,BONGARÁ,JUMBILLA,POINT (190075.874 9346359.715),POINT (189187.025 9294846.388),51.520995
4,AMAZONAS,CHACHAPOYAS,RODRIGUEZ DE MENDOZA,MENDOZA,POINT (225448.752 9292567.08),POINT (189187.025 9294846.388),36.333292
...,...,...,...,...,...,...,...
191,TUMBES,TUMBES,ZARUMILLA,ZARUMILLA,POINT (-86526.726 9611595.721),POINT (-107268.982 9603509.416),22.262739
192,UCAYALI,PUCALLPA,ATALAYA,ATALAYA,POINT (636058.908 8813886.747),POINT (550781.38 9073433.117),273.196952
193,UCAYALI,PUCALLPA,PADRE ABAD,AGUAYTÍA,POINT (443935.142 9001007.633),POINT (550781.38 9073433.117),129.079701
194,UCAYALI,PUCALLPA,CORONEL PORTILLO,PUCALLPA,POINT (551671.316 9073793.294),POINT (550781.38 9073433.117),0.960059


In [87]:
dt_final[dt_final['region']=='LAMBAYEQUE']

,region,capital,Provincia,Nombre,cap_provincia,cap_region,distancia_kilometros
124,LAMBAYEQUE,CHICLAYO,LAMBAYEQUE,LAMBAYEQUE,POINT (-42937.179 9256494.023),POINT (-35320.086 9248828.115),10.806769
125,LAMBAYEQUE,CHICLAYO,FERRENAFE,FERREÑAFE,POINT (-29974.985 9263573.87),POINT (-35320.086 9248828.115),15.684623
126,LAMBAYEQUE,CHICLAYO,CHICLAYO,CHICLAYO,POINT (-35144.309 9248672.918),POINT (-35320.086 9248828.115),0.234486


In [88]:
gdf[gdf['Departa']=='LORETO']

['CO230',	'Capital',	'CIUDAD',	'SAN ANTONIO DEL ESTRECHO',	'PUTUMAYO',	'LORETO',	-72.66846,	-2.45028,	1607,	16,	POINT (-72.66846 -2.45028)]

NameError: name 'POINT' is not defined